# Plane Laminate: Model Development Report

---

This notebook documents the devlopment of the `plane_laminate` model.

In [ ]:
import ipysheet
import grama as gr
import numpy as np
import pandas as pd

## Responses
---

### Verification

## Input distribution

---

The CFRP material property distribution is informed by data from Table 2 of Zhang et al.[1], but primarily comes from Table 6 of del Rosario et al.[2].

### Data Sources

#### Table 2 (Zhang et al.)

In [ ]:
df_zhang_t2 = pd.read_csv("zhang2019_table2.csv")
sheet = ipysheet.from_dataframe(df_zhang_t2)
display(sheet)

Using these, we can fix a few of the parameters immediately.

In [ ]:
## Mean quantities
E_xx_t_mu_zhang = 115e9  # Tensile modulus, [0] laminate
S_xx_t_mu_zhang = 1420e6 # Tensile strength, [0] laminate
S_xx_c_mu_zhang = 712e6 # Compressive strength, [0] laminate

## COV
# Assume the \pm quantities are symmetric normal 95% CI bounds
E_xx_t_cov_zhang =  2e9 / 1.96 / E_xx_t_mu_zhang # Tensile modulus, [0] laminate
S_xx_t_cov_zhang = 83e6 / 1.96 / S_xx_t_mu_zhang # Tensile strength, [0] laminate
S_xx_c_cov_zhang = 47e6 / 1.96 / S_xx_c_mu_zhang # Compressive strength, [0] laminate

These were reportedly the basis for our stample statistics for our Aviation paper [2], but I do not see all the reported values in the original Zhang et al. reference. Rick must have brought in some additional information he has as an employee of Arevo.

#### Table 6 (del Rosario et al.)

Aviation paper [2] Table 6 reproduced below. This is the core reference for the parameter data.

In [ ]:
df_zdr_t6 = pd.read_csv("zdr2019_table6.csv")
sheet = ipysheet.from_dataframe(df_zdr_t6)
display(sheet)

In [ ]:
## Copy from table
S_xx_mu_zdr = 1.42e9
S_xy_mu_zdr = 61.93e6

E_xx_mu_zdr = 114.9e9
E_yy_mu_zdr = 7.10e9
G_xy_mu_zdr = 4.21e9
nu_xy_mu_zdr = 0.45

S_xx_cov_zdr = 0.059
S_xy_cov_zdr = 0.072

E_xx_cov_zdr = 0.018
E_yy_cov_zdr = 0.085
G_xy_cov_zdr = 0.105
nu_xy_cov_zdr = 0.08

We have multiple sources for some of the values; let's compare those:

In [ ]:
df_compare = pd.DataFrame(
    data = {
        "source": ['zhang', 'zdr'],
        "S_xx_mu": [S_xx_t_mu_zhang, S_xx_mu_zdr],
        "E_xx_mu": [E_xx_t_mu_zhang, E_xx_mu_zdr],
        "S_xx_cov": [S_xx_t_cov_zhang, S_xx_cov_zdr],
        "E_xx_cov": [E_xx_t_cov_zhang, E_xx_cov_zdr]
    }
)
df_compare

It appears the nominal values are nearly identical, up to rounding. The COV are different; Rick must have made different assumptions on the \pm values than what I state above.

#### Performance Composites Table

These data are from the now-defunct company ["Performance Composites"](http://www.performance-composites.com/carbonfibre/mechanicalproperties_2.asp), accessed on 2019-11-28. While certainly the least trustworthy of the data reported here, this is the only reference I found on the transverse strength properties of unidirectional carbon fiber.

In [ ]:
df_pc_table = pd.read_csv("performance_composites_acc2019-11-28.csv")
sheet = ipysheet.from_dataframe(df_pc_table)
display(sheet)

The Standard Carbon Fiber Uni-Directional (`Std CF UD`) data imply the ratio between longitudinal and transverse tensile strengths can be on the order of `1500 / 50 = 30`. The loss for compressive strength is relatively lower at `1200 / 250 = 4.8`.

### Parameter Decisions

Table 6 provides 1st and 2nd moment information for the orthotropic material properties `E_xx, E_yy, G_xy, \nu_xy`. Table 6 does not report compressive modulii (as does Table 2); we assume the same elastic behavior in tension and compression.

Table 6 also provides 1st and 2nd moment information for two (tensile `xx` and shear `xy`) of the five (incl. compressive `xx`, tensile and compressive `yy`) failure modes we need. Thus we need to make decisions about the remaining failure modes.

The Performance Composite Table implies the ratio of longitudinal `S_xx` and transverse `S_yy` tensile strengths can be on the order of `30`; we will set the nominal tensile strengths appropriately and keep COV identical between `S_xx` and `S_yy`. The strength ratio for the compressive mode is `4.8`.

Table 2 gives us a ratio between compressive and tensile strength; we will use this ratio to inform the compressive strength based on tensile values from Table 6. We will use an identical COV between tensile and compressive modes. 

I am unaware of **any** correlation data between material properties of carbon fiber. I will set default correlation values to zero, but this is *only* based on a lack of data.

In [ ]:
## Strength properties
S_xx_t_mu = S_xx_mu_zdr
S_xx_c_mu = S_xx_t_mu * (S_xx_c_mu_zhang / S_xx_t_mu_zhang)
S_xy_s_mu = S_xy_mu_zdr 
S_yy_t_mu = S_xx_t_mu / 30
S_yy_c_mu = S_xx_c_mu / 4.8

S_xx_t_cov = S_xx_cov_zdr 
S_xx_c_cov = S_xx_cov_zdr 
S_xy_s_cov = S_xy_cov_zdr 
S_yy_t_cov = S_xx_cov_zdr 
S_yy_c_cov = S_xx_cov_zdr 

## Elastic properties
E_xx_mu = E_xx_mu_zdr 
E_yy_mu = E_yy_mu_zdr 
G_xy_mu = G_xy_mu_zdr 
nu_xy_mu = nu_xy_mu_zdr 

E_xx_cov = E_xx_cov_zdr 
E_yy_cov = E_yy_cov_zdr 
G_xy_cov = G_xy_cov_zdr 
nu_xy_cov = nu_xy_cov_zdr 

## Geometry properties
theta_pm = 3 * np.pi / 180
t_nom = 1e-3
t_pm = t_nom * 0.01
t_cov = t_pm / t_nom / np.sqrt(3)

## Loading properties
Nx_mu = 1.2e6
Nx_cov = 0.01

## Summarize in table
df_param = pd.DataFrame(
    data = {
        'property': ['E_xx', 'E_yy', 'nu_xy', 'G_xy', 'theta', 't', 'S_xx_t', 'S_yy_t', 'S_xx_c', 'S_yy_c', 'S_xy_s', 'Nx'],
        'mean': [E_xx_mu, E_yy_mu, nu_xy_mu, G_xy_mu, 0, t_nom, S_xx_t_mu, S_yy_t_mu, S_xx_c_mu, S_yy_c_mu, S_xy_s_mu, Nx_mu],
        'cov': [E_xx_cov, E_yy_cov, nu_xy_cov, G_xy_cov, 0, t_cov, S_xx_t_cov, S_yy_t_cov, S_xx_c_cov, S_yy_c_cov, S_xy_s_cov, Nx_cov]
    }
)
df_param

### Shape Decisions

## Bibliography

---

[1] Zhang, D., Rudolph, N., and Woytowitz, P., “Reliable Optimized Structures with High Performance Continuous Fiber Thermoplastic Composites from Additive Manufacturing (AM),” SAMPE 2019 , Charlotte, NC, May 2019.

[2] del Rosario, Z., Fenrich, R., and Iaccarino, G. "Margin as Model: Some Answers to "How Many Tests Should I Perform?"," AIAA Aviation 2019, Dallas, TX, June 2019.